In [1]:
import time
from os import environ

from spatialyze.database import database
from spatialyze.predicate import camera, objects, lit
from spatialyze.utils import F
from spatialyze.utils.process_raw_nuscenes import process_raw_nuscenes

In [2]:
database.load_roadnetworks('../../data/scenic/road-network/boston-seaport', 'boston-seaport')

In [3]:
database.reset()

In [4]:
annotations, cameras = process_raw_nuscenes('/workspace/data/nuscenes/mini')

Loaded json files
Processing sample data


100%|███████████████████████████████████████████████████████████████████████████████████| 14008/14008 [00:02<00:00, 5118.74it/s]


Processing sample annotation


100%|██████████████████████████████████████████████████████████████████████████████████| 18538/18538 [00:00<00:00, 50608.91it/s]


Assigning annotations to camera


100%|████████████████████████████████████████████████████████████████████████████████████| 18538/18538 [00:34<00:00, 531.20it/s]


Indexing Cameras by Camera Id


100%|██████████████████████████████████████████████████████████████████████████████████| 14008/14008 [00:00<00:00, 54394.47it/s]


Indexing Annotations by Camera Id


100%|█████████████████████████████████████████████████████████████████████████████████| 18538/18538 [00:00<00:00, 187672.90it/s]


In [5]:
keys = list(annotations.keys())
key = [*filter(lambda k: k.scene=='scene-0103' and k.channel=='CAM_FRONT', keys)][0]
key

CameraKey(scene='scene-0103', channel='CAM_FRONT')

In [6]:
camera = cameras[key]
annotation = annotations[key]

In [7]:
camera

[NuscenesCamera(token='4f5e35aa6c6a426ca945e206fb2f4921', sample_token='3e8750f331d7499e9b5123e9eb70f2e2', timestamp=1533151603512404, is_key_frame=True, filename='samples/CAM_FRONT/n008-2018-08-01-15-16-36-0400__CAM_FRONT__1533151603512404.jpg', sample_timestamp=1533151603547590, camera_translation=array([6.01382107e+02, 1.64679996e+03, 1.46440790e+00]), camera_rotation=array([-0.36717878,  0.36000235, -0.61227966,  0.60057611]), camera_intrinsic=[[1252.8131021185304, 0.0, 826.588114781398], [0.0, 1252.8131021185304, 469.9846626224581], [0.0, 0.0, 1.0]], ego_translation=[599.849775495386, 1647.6411294309523, 0.0], ego_rotation=[-0.9687876119182126, -0.004506968075376869, -0.00792272203393983, 0.24772460658591755], scene_name='scene-0103', channel='CAM_FRONT', location='boston-seaport', ego_heading=-118.69178864587704, camera_heading=-118.10389082098091, frame_order=2),
 NuscenesCamera(token='728e987b139c481597bf032a47a30d39', sample_token='3950bd41f74548429c0f7700ff3d8269', timestamp=

In [8]:
database.load_nuscenes(annotations, cameras)

Ingesting Cameras and Annotations


100%|███████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:04<00:00,  4.85it/s]


## Simple Query

In [9]:
c = camera
o = objects[0]
output = database.predicate(o.type == 'vehicle.car')

In [10]:
len(output)

19725

## Figure 12

In [13]:
t0 = time.time()
o = objects[0]
c = camera
pred1 = (
    # (F.like(c.filename, 'samples/CAM_FRONT%')) & 
    (o.type == 'human.pedestrian.adult') &
    # F.contained(c.ego, 'intersection') &
    F.contains(o.trans@c.time, 'intersection') &
    F.angle_excluding(F.facing_relative(o.trans@c.time, c.cam), lit(-70), lit(70)) &
    # F.angle_between(F.facing_relative(c.cam, F.road_direction(c.ego)), lit(-15), lit(15)) &
    (F.distance(c.ego, o.trans@c.time) < lit(50)) & # &
    (F.view_angle(o.trans@c.time, c.cam) < lit(35))
)
output = database.predicate(pred1)
t1 = time.time()

AttributeError: module 'spatialyze.utils.F' has no attribute 'angle_excluding'

In [ ]:
print(len(output), t1 - t0)

## Figure 13

In [ ]:
t0 = time.time()
obj1 = objects[0]
obj2 = objects[1]
cam = camera
pred2 =(
    # (F.like(cam.filename, 'samples/CAM_FRONT%')) & 
    (obj1.id > obj2.id) &
    ((obj1.type == 'vehicle.car') | (obj1.type == 'vehicle.truck')) &
    ((obj2.type == 'vehicle.car') | (obj2.type == 'vehicle.truck')) &
    # F.angle_between(F.facing_relative(cam.cam, F.road_direction(cam.cam)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
    (F.view_angle(obj1.trans@cam.time, cam.cam) < lit(35)) &
    (F.distance(cam.ego, obj2.trans@cam.time) < 50) &
    (F.view_angle(obj2.trans@cam.time, cam.cam) < lit(35)) &
    F.contained(obj1.trans@cam.time, 'intersection') &
    F.contained(obj2.trans@cam.time, 'intersection') &
    # F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.cam), 40, 135) &
    # F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.cam), -135, -50) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), -180, -90)
    # (F.min_distance(cam.ego, 'intersection') < 10) &
    # F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100)
)
output = database.predicate(pred2)
t1 = time.time()

In [ ]:
len(output)

In [ ]:
t1 - t0

## Figure 14

In [ ]:
t0 = time.time()
obj1 = objects[0]
cam = camera
pred3 = (
    # (F.like(cam.filename, 'samples/CAM_FRONT%')) & 
    ((obj1.type == 'vehicle.car') | (obj1.type == 'vehicle.truck')) &
    (F.view_angle(obj1.trans@cam.time, cam.cam) < lit(35)) &
    F.angle_between(F.facing_relative(cam.cam, F.road_direction(cam.ego, cam.ego)), 135, 225) &
    F.contained(cam.cam, F.road_segment('lane')) &
    F.contained(obj1.trans@cam.time, F.road_segment('lane')) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, F.road_direction(obj1.trans@cam.time, cam.ego)), -15, 15) &
    # F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 135, 225) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 10)
)
output = database.predicate(pred3)
t1 = time.time()

In [ ]:
print(len(output), t1 - t0)

## Figure 15

In [ ]:
t0 = time.time()
cam = camera
car1 = objects[0]
opposite_car_1 = objects[1]
opposite_car_2 = objects[2]

pred4 = (
    # (F.like(cam.filename, 'samples/CAM_FRONT%')) & 
    ((car1.type == 'vehicle.car') | (car1.type == 'vehicle.truck')) &
    ((opposite_car_2.type == 'vehicle.car') | (opposite_car_2.type == 'vehicle.truck')) &
    ((opposite_car_1.type == 'vehicle.car') | (opposite_car_1.type == 'vehicle.truck')) &
    (opposite_car_1.id > opposite_car_2.id) &
    (car1.id < opposite_car_2.id) &

    F.contained(cam.cam, F.road_segment('lane')) &
    F.contained(car1.trans@cam.time, F.road_segment('lane')) &
    F.contained(opposite_car_1.trans@cam.time, F.road_segment('lane')) &
    F.contained(opposite_car_2.trans@cam.time, F.road_segment('lane')) &
    F.angle_between(F.facing_relative(cam.cam, F.road_direction(cam.cam, cam.cam)), -15, 15) &
    (F.view_angle(car1.trans@cam.time, cam.cam) < lit(35)) &
    (F.distance(cam.cam, car1.trans@cam.time) < 40) &
    F.angle_between(F.facing_relative(car1.trans@cam.time, cam.ego), -15, 15) &
    # F.angle_between(F.facing_relative(car1.trans@cam.time, F.road_direction(car1.trans@cam.time, cam.ego)), -15, 15) &
    F.ahead(car1.trans@cam.time, cam.cam) &
    # (F.convert_camera(opposite_car.trans@cam.time, cam.ego) > [-10, 0]) &
    # (F.convert_camera(opposite_car.trans@cam.time, cam.ego) < [-1, 50]) &
    F.angle_between(F.facing_relative(opposite_car_1.trans@cam.time, cam.ego), 135, 225) &
    # (F.distance(opposite_car@cam.time, car2@cam.time) < 40)# &
    F.angle_between(F.facing_relative(opposite_car_2.trans@cam.time, opposite_car_1.trans@cam.time), -15, 15) &
    F.angle_between(F.facing_relative(opposite_car_2.trans@cam.time, F.road_direction(opposite_car_2.trans@cam.time, cam.ego)), -15, 15) &
    F.ahead(opposite_car_2.trans@cam.time, opposite_car_1.trans@cam.time)
    )
output = database.predicate(pred4)
t1 = time.time()

In [ ]:
print(len(output), t1 - t0)